In [25]:
import requests
from lxml import html
import smtplib, ssl
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from datetime import datetime
from google.colab import files
import pytz



In [26]:
def check_price(URL):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'Accept-Encoding':  None
    }
    
    page = requests.get(URL,headers=headers)
    tree = html.fromstring(page.content)
    
    # open devolper option for amazon and note the class name or id name for the price
    
    price = tree.xpath('//span[@class="a-price-whole"]/text()')[0]
    return int(float(price.replace(',','')))


In [27]:
def email_price(url,price):
    
    # please read this. As less securtiy option is not working in google. you have to on two step verification and next go to App password and set a password
    
    # add the mail and password
    smtp_server = "smtp.gmail.com"
    port = 587  # For starttls
    
    #Enter your mail id
    sender_email = ""
    
    # Enter your password
    password = ""

    # Create a secure SSL context
    context = ssl.create_default_context()
    pm=pro_name(url)
    # Try to log in to server and send email
    try:
        server = smtplib.SMTP(smtp_server, port)
        server.ehlo()  # Can be omitted
        server.starttls(context=context)  # Secure the connection
        server.ehlo()  # Can be omitted
        server.login(sender_email, password)
        subject = "Price fell down!"
        body=url
        message= f"Subject: {subject}\n\n{pm}\n\n{body}\n\n present price is {price}"
        server.sendmail(sender_email,sender_email,message)

    except Exception as e:
        # Print any error messages to stdout
        print(e)
    finally:
        server.quit()



In [28]:
def pro_name(url):
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
      'Accept-Encoding': None
  }     
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  product_name = soup.find("span", {"id": "productTitle"}).text.strip()
  return product_name



In [29]:

def write_to_csv(product_name, price):
    filename = "/content/drive/MyDrive/{}.csv".format(product_name)
    headers = ['Date', 'Time', 'Price']

    india_tz = pytz.timezone('Asia/Kolkata')
    current_time = datetime.now(india_tz)

    rows = [[datetime.now().strftime("%Y-%m-%d"), current_time.strftime("%H:%M:%S"), price]]
    # if the file does not exist, create a new file with headers
    try:
        with open(filename, 'x') as f:
            writer = csv.writer(f)
            writer.writerow(headers)
            writer.writerows(rows)
    except FileExistsError:
        # if the file already exists, append new data to it
        with open(filename, 'a') as f:
            writer = csv.writer(f)
            writer.writerows(rows)



In [30]:
def collect_data(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36',
        'Accept-Encoding': None
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    review_section = soup.find("div", {"id": "reviews-medley-footer"})
    review_links = review_section.find_all("a")

    for link in review_links:
        href = link.get("href")
        if href.startswith("http" + "&pageNumber="):
            url1 = href + "&pageNumber="
        else:
            url1 = "https://www.amazon.in" + href + "&pageNumber="
    # creating the dataframe
    cust_name = []
    rev_date = []
    ratings = []
    rev_title = []
    rev_content = []
    for page in range(1, 35):
        new_url = url1 + str(page)
        code = requests.get(new_url, headers=headers)
        if str(code) == "<Response [200]>":
            soup = BeautifulSoup(code.content, 'html.parser')
            names = soup.select('span.a-profile-name')[2:]
            titles = soup.select('a.review-title span')
            dates = soup.select('span.review-date')[2:]
            stars = soup.select('i.review-rating span.a-icon-alt')[2:]
            reviews = soup.select('span.review-text-content span')
            h = min(len(titles), len(dates), len(stars), len(reviews))
            for i in range(h):
                cust_name.append(names[i].get_text())
                rev_date.append(dates[i].get_text().replace("Reviewed in India on ", ""))
                ratings.append(stars[i].get_text().replace("out of 5 stars",""))
                rev_title.append(titles[i].get_text())
                rev_content.append(reviews[i].get_text().strip("\n "))
    df = pd.DataFrame()
    df['Customer Name'] = cust_name
    df['Date'] = rev_date
    df['Ratings'] = ratings
    df['Review Title'] = rev_title
    df['Reviews'] = rev_content
    df.to_csv('amazon.csv')
    from google.colab import files
    files.download('amazon.csv')
    


In [31]:
def repl(pm):
    pm=pm.replace(" ","_")
    pm=pm.replace("-","_")
    pm=pm.replace("/","_")
    pm=pm.replace("\\","_")
    return pm

In [ ]:
if __name__ == "__main__":
    url = "https://www.amazon.in/Apple-iPhone-128GB-Product-RED/dp/B0BDJVSDMY/ref=sr_1_1_sspa?crid=3BPUGJSPF8GK3&keywords=iphone+14&qid=1675792641&sprefix=iphone%2Caps%2C243&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"
    collect_data(url)
    pm = pro_name(url)
    pm=repl(pm)
    print(pm)
    i=0
    while(True):
        price = check_price(url)
        if (price) < 80000:
            email_price(url,price)
            print(price)
            print("The mail is sent")
            write_to_csv(pm, price)
        else:
            print("the price did not fell")
            write_to_csv(pm, price)
        i=i+1
        time.sleep((60*60*4))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Apple_iPhone_14_128GB_(Product)_RED
72499
The mail is sent
